# Лабораторная работа по теме: "Сбор и подготовка данных с помощью языка Python"

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sql

In [ ]:
pip install odfpy

*Дополнительно выполнил практические задания по 3му модулю, их можно найти по [ссылке](https://colab.research.google.com/drive/1uw7mJf45TnFahjjrhboyBEd1GwXFQ9VF?usp=sharing). Если у Вас есть какие-то интеерсные ресурсы, где есть ещё практические задания по данной теме, оставьте пожалуйста ссылку в комментарии к этой записи. Тема понравилась, было бы интересно сделать по ней что-то ещё.

### Задание №1

**Условие**: Для заданий этого занятия требуется загрузить датасет 'Employee_Salary_Dataset.ods' ([страница датасета на Kaggle](https://www.kaggle.com/datasets/anninasimon/employee-salary-dataset)). Датасет можно загрузить из директории datasets. Нужные данные находятся на листе 'Sheet1'. В датасете содержатся срез данных о зарплатах различных сотрудников. Загрузите эти данные в виде датафрейма.

Воспользуемся gdown, чтобы иметь возможность подгружать файлы с расширением .ods (хочется иметь возможность загружать файл из внешнего источника, чтобы код мог запустить кто угодно). Таким образом, указав `gdown.download(salary_url, output_path, quiet=False)` мы загрузим файл, который находится по адресу, указанному в переменной `salary_url`. Этот файл будет сохранён в текущую рабочую директорию под именем, указанном в `output_path`. Также, для удобства, введём ещё один аргумент `quiet=False`, чтобы отображать в консоли статус загрузки и удостовериться в том, что файл успешно был обработан.

In [ ]:
import gdown # нужен для загрузки ods файлов

# Загрузка данных из файла .ods
salary_url = "https://github.com/MatNepo/DataFramesPython/raw/main/datasets/Employee_Salary_Dataset.ods"
output_path = "Employee_Salary_Dataset.ods"
gdown.download(salary_url, output_path, quiet=False)

# Загрузка данных в виде датафрейма
df_salary = pd.read_excel(output_path, engine='odf', sheet_name='Sheet1', index_col=0)
df_salary

Downloading...
From: https://github.com/MatNepo/DataFramesPython/raw/main/datasets/Employee_Salary_Dataset.ods
To: /content/Employee_Salary_Dataset.ods
100%|██████████| 14.7k/14.7k [00:00<00:00, 22.2MB/s]


,Experience_Years,Age,Gender,Salary
ID,,,,
1,5,28,Female,250000
2,1,21,Male,50000
3,3,23,Female,170000
4,2,22,Male,25000
5,1,17,Male,10000
6,25,62,Male,5001000
7,19,54,Female,800000
8,2,21,Female,9000
9,10,36,Female,61500


### Задание №2

**Условие**: Выведите общую информацию об этом датафрейме.

In [ ]:
df_salary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 1 to 35
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Experience_Years  35 non-null     int64 
 1   Age               35 non-null     int64 
 2   Gender            35 non-null     object
 3   Salary            35 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.4+ KB


### Задание №3

**Условие**: Приведите все названия столбцов к нижнему регистру.

Воспользуемся атрибутом `columns`, чтобы получить массив, состоящий из названий столбцов, и методом `str.lower`, чтобы привести их названия к нижнему регистру. Полуим `columns.str.lower`. Теперь выведем начало DataFrame'а при помощи метода `head()` (весь DataFrame выводить ене надо, т. к. в этом задании интересуют только названия столбцов).

*по умолчанию он выводит первые 5 строк

In [ ]:
df_salary_lower_case = df_salary.copy()
df_salary_lower_case.head().columns.str.lower()
df_salary_lower_case.head()

,Experience_Years,Age,Gender,Salary
ID,,,,
1,5,28,Female,250000
2,1,21,Male,50000
3,3,23,Female,170000
4,2,22,Male,25000
5,1,17,Male,10000


### Задание №4*

Удалите выбросы по зарплате (salary) при помощи IQR3 (тройной межквартильный интервал). Сохраните в отдельном датафрейме срез без выбросов.

1) Вычислим IQR3:

* IQR3 – это метод определения выбросов, основанный на межквартильном размахе.
* Границы выбросов по IQR3: Q1 - 3 * IQR и Q3 + 3 * IQR.
* Датафрейм df_salary_filtered содержит только те записи, где зарплата находится в пределах этих границ.

In [ ]:
# Вычисление квартилей
Q1 = df_salary['Salary'].quantile(0.25)
Q3 = df_salary['Salary'].quantile(0.75)

# Вычисление IQR
IQR = Q3 - Q1

# Вычисление границы выбросов
iqr_3_lower = Q1 - 3 * IQR
iqr_3_upper = Q3 + 3 * IQR

2) Отфильтровка данных:

* `df_salary['Salary'] > iqr_3_lower` - создает последовательность булевых значений, где True соответствует значениям 'Salary', которые больше нижней границы IQR3.
* `df_salary['Salary'] < iqr_3_upper` - аналогично, создает булеву серию, где True соответствует значениям 'Salary', которые меньше верхней границы IQR3.
* `... & ...` - объединим два последних выражения, тем самым сформировав необходимый интервал значений внутри IQR3
* `df_salary.loc[...]` - используется для выбора строк из датафрейма, соответствующих условиям внутри .loc[]. В результате получим новый датафрейм, содержащий только те строки, где 'Salary' находится в пределах IQR3.

In [ ]:
# Создание среза без выбросов
df_salary_no_outliers = df_salary[(df_salary['Salary'] >= iqr_3_lower) & (df_salary['Salary'] <= iqr_3_upper)]

3) Выведем результат

Выведем первые несколько строк полученного среза (при желании вывода полной таблицы - убрать .head())

In [ ]:
# Вывести первые несколько строк датафрейма без выбросов
df_salary_no_outliers.head()

,Experience_Years,Age,Gender,Salary
ID,,,,
1,5,28,Female,250000
2,1,21,Male,50000
3,3,23,Female,170000
4,2,22,Male,25000
5,1,17,Male,10000


### Задание №5

**Условие**: Из [страницы](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fru.wikipedia.org%2Fwiki%2F%25D0%2594%25D0%25BE%25D1%2585%25D0%25BE%25D0%25B4%25D1%258B_%25D0%25BD%25D0%25B0%25D1%2581%25D0%25B5%25D0%25BB%25D0%25B5%25D0%25BD%25D0%25B8%25D1%258F_%25D0%25A0%25D0%25BE%25D1%2581%25D1%2581%25D0%25B8%25D0%25B8) в википедии получите таблицу с данными о среднемесячных доходах в субъектах Сибирского ФО.

В этой работе, чтобы усложнить задачу, зададим условие, что нету файла с таблицей, который можно загрузить, есть только ссылка на сторонний ресурс, в данном случае, [Wikipedia](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fru.wikipedia.org%2Fwiki%2F%25D0%2594%25D0%25BE%25D1%2585%25D0%25BE%25D0%25B4%25D1%258B_%25D0%25BD%25D0%25B0%25D1%2581%25D0%25B5%25D0%25BB%25D0%25B5%25D0%25BD%25D0%25B8%25D1%258F_%25D0%25A0%25D0%25BE%25D1%2581%25D1%2581%25D0%25B8%25D0%25B8).

1. Загрузка библиотек

Начнём с подгрузки таких необходимых библиотек как `requests` и `bs4`.

* `requests` -  используется для отправки HTTP-запроса на веб-страницу Википедии и получения HTML-кода страницы.
* `bs4` - используется для парсинга HTML-кода веб-страницы и извлечения информации из таблицы о среднемесячных доходах в субъектах Сибирского Федерального округа с веб-страницы Википедии.

In [ ]:
import requests
from bs4 import BeautifulSoup


2. Загрузка и парсинг веб-страницы

С помощью `requests.get(url)` выполняим HTTP GET-запрос к указанному URL и возвратим объект `response`.
```
url = "link"
response = requests.get(url)
```
Тут же выполним проверку на успешность выполнения запроса `response.status_code == 200?`. Если придёт True, то запрос был выполнен успешен и теперь можно получить HTML-код страницы: `html_code = response.text`.
```
if response.status_code == 200:
    print("Запрос выполнен успешно.")
    html_code = response.text
```
В случае получения False выведем значение ошибки `response.status_code`.
```
else:
    print(f"Ошибка при выполнении запроса. Статус код: {response.status_code}")
```
Теперь можем произвести парсинг HTML-кода страницы: `soup = BeautifulSoup(response.text, 'html.parser')`.
```
soup = BeautifulSoup(response.text, 'html.parser')
```

In [ ]:
# Загрузка страницы Википедии
url = "https://ru.wikipedia.org/wiki/%D0%94%D0%BE%D1%85%D0%BE%D0%B4%D1%8B_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"
response = requests.get(url)

# Проверка статуса ответа
if response.status_code == 200:
    # Если статус ответа 200 (OK), это означает успешный запрос
    print("Запрос выполнен успешно.")
    # Получение HTML-кода страницы
    html_code = response.text

else:
    # Если статус ответа не 200, это может означать проблемы с запросом
    print(f"Ошибка при выполнении запроса. Статус код: {response.status_code}")

# Используем BeautifulSoup для парсинга HTML-кода страницы
soup = BeautifulSoup(response.text, 'html.parser')


Запрос выполнен успешно.


3. Поиск таблицы с данными в коде страницы

Чтобы найти таблицу воспользуемся ключевыми словами языка HTML: найдём первый элемент таблицы (`<table>`) с классом wikitable (`"class": "wikitable"`) в объекте BeautifulSoup (soup - древовидная структура с HTML кодом страницы). Таким образом, получаем следующий скрипт:
```
table = soup.find("table", {"class": "wikitable"})`
```

*первый элемент ищем как первое вхождение с использованием `soup.find()`

In [ ]:
# Находим таблицу с данными о среднемесячных доходах в субъектах Сибирского ФО
table = soup.find("table", {"class": "wikitable"})


4. Вывод датафрейма

Загрузим найденную таблицу в датафрейм с помощью `read_html(str(table))[0]`:
```
df_siberian_fo = pd.read_html(str(table))[0]
```
Выведем несколько первых строк датафрейма:

In [ ]:
# Загружаем таблицу в датафрейм
df_siberian_fo = pd.read_html(str(table))[0]

# Выводим первые несколько строк датафрейма
df_siberian_fo.head()

,Federal subject,RUB,USD
,Россия,62 270,684
0,Центральный федеральный округ,NaN,NaN
1,Москва,113 671,1249
2,Белгородская область,46 133,506
3,Брянская область,40 354,443
4,Владимирская область,44 188,485


### Задание №6

**Условие**: Запишите загруженный датафрейм с доходами в виде таблицы incomes внутри файла sqlite в директории с блокнотом. Не сохраняйте индекс, в случае, если название таблицы уже есть в БД - перепишите эту таблицу.

Подгрузим библитотеку `sqlalchemy`, которая позволит нам локально создать базу данных. Так как из этой библиотеки нам нужен будет только метод, который непосредственно отвечает за создание базы данных, то можем загрузить только его (`create_engine`)

In [ ]:
from sqlalchemy import create_engine

Сооздадим объект `engine` для подключения к базе данных SQLite:

In [ ]:
engine = create_engine('sqlite:///my_database.db')

In [ ]:
# Выводим данные из таблицы 'incomes' с использованием контекстного менеджера
with engine.connect() as conn:
    # Выбираем все данные из таблицы 'incomes' в виде датафрейма
    df_from_sql = pd.read_sql_table('incomes', conn)

df_from_sql

,"('Federal subject', 'Россия')","('RUB', '62 270')","('USD', '684')"
0,Центральный федеральный округ,None,None
1,Москва,113 671,1249
2,Белгородская область,46 133,506
3,Брянская область,40 354,443
4,Владимирская область,44 188,485
...,...,...,...
88,Амурская область,67 992,747
89,Магаданская область,123 813,1 360
90,Сахалинская область,105 614,1 160
91,Еврейская автономная область,56 083,616
